<a href="https://colab.research.google.com/github/data04190/News_Classification/blob/main/baseline_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

  Using cached mxnet-1.9.0-py3-none-manylinux2014_x86_64.whl (47.3 MB)
  Using cached graphviz-0.8.4-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.5 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595725 sha256=3459d62cda2bd920ea307c0192637604940b048ec6b84145431cdfdcd430aa10
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 769 kB 5.3 MB/s 
     |████████████████████████████████| 3.0 MB 38.9 MB/s 
     |████████████████████████████████| 895 kB 43.6 MB/s 


In [ ]:
# KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-g7t2_zlq
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-g7t2_zlq
     |████████████████████████████████| 131 kB 5.2 MB/s 
     |████████████████████████████████| 4.5 MB 42.5 MB/s 
     |████████████████████████████████| 3.4 MB 36.1 MB/s 
     |████████████████████████████████| 596 kB 48.4 MB/s 
     |████████████████████████████████| 3.3 MB 38.8 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 79 kB 7.0 MB/s 
     |████████████████████████████████| 8.5 MB 21.6 MB/s 
     |████████████████████████████████| 127 kB 45.4 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15449 sha256=fdd8efed8ec43dfbaffc84228fa5fbb0edf78f5fd217677038a52349ef70373d
  Stored in directory: /tmp/pip-ephem-wheel-cache-owz1uvgv/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd05

In [ ]:
#라이브러리 불러오기
import os
import pandas as pd
import numpy as np
import random
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp

from datetime import datetime, timezone, timedelta
import torch.optim as optim
from sklearn.metrics import f1_score, accuracy_score
from tqdm.notebook import tqdm

# koBert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 시드 고정
RANDOM_SEED = 2022
torch.manual_seed(RANDOM_SEED)    #random seed 고정
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 경로 설정
ROOT_PATH ='/content/drive/MyDrive/AI_hub/data/' ## 코드와 데이터가 위치한 경로 지정 #####
DATA_DIR = '/content/drive/MyDrive/AI_hub/data/' # 코드와 데이터가 위치한 경로 지정 #####

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')

In [ ]:
train.head()

,text,category
0,이에 따른다면 초등학생들의 소비 문화는 디지털 강국의 ‘개국 공신’이 되려는 몸부림...,연예/문화
1,"""지난 14일 우리정부가 난민 신청을 한 예멘인들에 대해 인도적 체류를 허가했는데요...",연예/문화
2,인천시 서구와 계양구에서 경기도 부천을 거쳐 서울 홍대까지 지하철이 건설될 것으로 ...,생활
3,김성태 자유한국당 의원이 최근 황운하 울산경찰청장과 벌인 ‘미친개’ 공방을 언급하며...,정치
4,"""지난해 사상 최대 실적을 거뒀던 우리 수출이 새해에도 좋은 흐름을 이어갔습니다. ...",스포츠


In [ ]:
# 학습 데이터의 카테고리 분포
train.groupby('category').count()

,text
category,
경제,3815
사회,5784
생활,5066
스포츠,4499
연예/문화,10830
정치,7654


In [ ]:
#카테고리형 데이터(Categorical Data)를 수치형 데이터(Numerical Data)로 변환

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()   
encoder.fit(train['category'])
train['category'] = encoder.transform(train['category'])   
train.head()

,text,category
0,이에 따른다면 초등학생들의 소비 문화는 디지털 강국의 ‘개국 공신’이 되려는 몸부림...,4
1,"""지난 14일 우리정부가 난민 신청을 한 예멘인들에 대해 인도적 체류를 허가했는데요...",4
2,인천시 서구와 계양구에서 경기도 부천을 거쳐 서울 홍대까지 지하철이 건설될 것으로 ...,2
3,김성태 자유한국당 의원이 최근 황운하 울산경찰청장과 벌인 ‘미친개’ 공방을 언급하며...,5
4,"""지난해 사상 최대 실적을 거뒀던 우리 수출이 새해에도 좋은 흐름을 이어갔습니다. ...",3


In [ ]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: '경제', 1: '사회', 2: '생활', 3: '스포츠', 4: '연예/문화', 5: '정치'}

In [ ]:
data_list = []
for q, label in zip(train['text'], train['category'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
data_list[:1]

[['이에 따른다면 초등학생들의 소비 문화는 디지털 강국의 ‘개국 공신’이 되려는 몸부림으로 여길 수도 있다. 하지만 지나친 부모의 교육열이 부른 무분별한 소비라는 지적도 만만치 않다. 경기도 수원의 한 초등학교 교사는 “수업 시간에 휴대전화로 문자를 주고받기에 혼냈더니 다음날 부모가 찾아와 ‘내 아이 문제에 끼어들지 말라’고 했다. 요즘 초등학생들은 서구적 가치관도 전통적 가치관도 지니지 못한 경우가 많다”고 고민을 토로했다. 사진 5 BOX ▲ 요즘 아프트 놀이터에서 초등학생을 찾기는 쉽지 않다. 지금 아이들은 학원과 PC방을 전전하고 있는지도 모른다. 요즘 취학하는 아이들은 심각한 주의력 결핍증이 있다고 한다. 유아기 때부터 온갖 시각 영상물에 노출된 탓이다. 다양한 시각 정보를 처리하는 데 익숙하더라도 그 속에서 소비 욕구만 자극받는다면 헛일이다. 인터넷 게임을 넓은 화면에서 즐기고 싶다며 벽걸이형 PDP(Plasma Display Panel) 텔레비전 구입을 요구하는 아이에게 무슨 말을 할 수 있을까. 언제까지 자녀와의 친밀도를 ‘경제력’으로 확인할 수도 없는 일 아닌가. 참교육을 위한 전국 학부모회 장은숙 사무처장은 “올바른 디지털 소비문화 기준을 세워서 적용해야 한다. 적어도 무엇 때문에 필요한지에 대해 ‘식탁 토론’이라도 벌이는 게 중요하다”고 말한다. 글 ▶[한겨레21]바로가기',
  '4']]

In [ ]:
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해서는 bert가 학습하지 않음(128->64로 변경)
batch_size = 32 #64->32로 변경

In [ ]:
class CustomDataset(Dataset):
   
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#train & val 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_val = train_test_split(data_list, test_size=0.1, random_state=42)

In [ ]:
print(len(dataset_train))
print(len(dataset_val))

33883
3765


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = CustomDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_val = CustomDataset(dataset_val, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=5)
valid_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, shuffle=True, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Hyper-parameters
learning_rate =  2e-5
num_epochs = 5
log_interval = 200
warmup_ratio = 0.1

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   #6개분류
                 dr_rate=None,
                 params=None):
        super(BertClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BertClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
criterion = 0 #initialization
print('Training Start...')

for e in range(num_epochs):
    train_acc = 0.0
    val_acc = 0.0
    model.train()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train loss {} train acc {}".format(e+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(valid_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        val_acc += calc_accuracy(out, label)
        
    print("epoch {} valid loss {} valid acc {}".format(e+1, loss.data.cpu().numpy(), val_acc / (batch_id+1)))
    if val_acc / (batch_id+1) > criterion:
        # 모델이 개선됨 -> 검증 점수와 weight 갱신
        criterion = val_acc / (batch_id+1)
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        # 모델이 개선됨 -> 모델 저장
        print("Update model save...")
        torch.save(check_point, os.path.join(ROOT_PATH, 'model_best.pt'))
        
print('Training Finish!')

Training Start...


  0%|          | 0/1059 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.8678971529006958 train acc 0.09375
epoch 1 batch id 201 loss 1.574506163597107 train acc 0.279384328358209
epoch 1 batch id 401 loss 0.6699297428131104 train acc 0.45807356608478805
epoch 1 batch id 601 loss 0.8202346563339233 train acc 0.5491368552412645
epoch 1 batch id 801 loss 0.9937913417816162 train acc 0.5965980024968789
epoch 1 batch id 1001 loss 1.0056946277618408 train acc 0.6302135364635365
epoch 1 train loss 0.5266179442405701 train acc 0.6380560801594796


  0%|          | 0/118 [00:00<?, ?it/s]

...

epoch 4 valid loss 0.5428845286369324 valid acc 0.8122099475383374
Update model save...


  0%|          | 0/1059 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.13938938081264496 train acc 0.96875
epoch 5 batch id 201 loss 0.3056948781013489 train acc 0.914179104477612
epoch 5 batch id 401 loss 0.28236833214759827 train acc 0.9156016209476309
epoch 5 batch id 601 loss 0.3001231253147125 train acc 0.9167533277870217
epoch 5 batch id 801 loss 0.15856200456619263 train acc 0.9176810237203495
epoch 5 batch id 1001 loss 0.49322018027305603 train acc 0.9172077922077922
epoch 5 train loss 0.30551978945732117 train acc 0.9169213181547932


  0%|          | 0/118 [00:00<?, ?it/s]

epoch 5 valid loss 0.5473942756652832 valid acc 0.8140763720742533
Update model save...
Training Finish!


In [ ]:
# 테스트 데이터셋 클래스 정의
class TestDataset(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.categories = mapping
        self.num_labels = len(self.categories)
        
        self.sentences = [transform([i]) for i in dataset]
        
    def label_decoder(self, labels):
        try:
            labels = list(map(lambda x : self.categories[x], labels))
            return labels
        except:
            assert 'Invalid intent'
            
    def __len__(self):
        return (len(self.sentences))
    
    def __getitem__(self, i):
        return (self.sentences[i])

In [ ]:
# 테스트 데이터 로드 및 출력 확인
test_data = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))
test_data.head(10)

,ID,text
0,0,"""[기자] 지난 24일 국회 국방위원회에서 현역 대령인 기무부대장이 송영무 국방장관..."
1,1,[IMG1] 한예슬-에릭 커플의 드라마 '스파이명월'의 시청률이 3회연속 하락세를 ...
2,2,"""디지털 광장입니다. 운전대를 잡는 시간보다 구르고 넘어지는 시간이 더 많은 극한의..."
3,3,정부가 최저 임금 인상 대책으로 자영업자와 영세사업자에게 지원하는 일자리 안정자금의...
4,4,청와대가 1박2일간 숨가빴던 방북 일정을 마치고 돌아온 대북특별사절단의 후일담을 8...
5,5,서울시는 올해 2000억원을 포함해 2022년까지 민관이 참여하는 혁신성장펀드 1조...
6,6,"""성폭행 의혹을 받는 안희정 전 충남지사가 오늘(8일) 오후에 예정됐던 기자회견을 ..."
7,7,"소지섭♡손예진 ‘지만갑’의 아역 김지환, 또 다른 숨은 주인공영화 ‘지금 만나러 갑..."
8,8,이이경(30)과의 열애를 인정한 배우 정인선(28)이 화제인 가운데 과거 이력도 주...
9,9,“불투명한 회계 문제가 원인 국책硏 이제 경각심 가져야” 청와대도 압력설 전면 부인...


In [ ]:
# 테스트 데이터 로드
dataset_test = test_data['text'].tolist()
data_test = TestDataset(dataset_test, 0, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
MODEL_DIR = os.path.join(ROOT_PATH, 'model_best.pt')
model = BertClassifier(bertmodel,  dr_rate=0.5).to(device)
model.load_state_dict(torch.load(MODEL_DIR)['model'])
""" 이전에 학습한 모델 weight파일을 불러 추론하려면 위의 주석을 풀고 실행
    학습 진행 후 바로 추론하는 경우 학습 과정의 model 사용 (주석 풀지 않고 실행) """

model.eval()
pred = []

for batch_id, (token_ids, valid_length, segment_ids) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length.long().to(device)

    out = model(token_ids, valid_length, segment_ids)
    pred.extend(out.argmax(dim=1).tolist())
    
    # 진행과정 출력
    if batch_id % 10 == 0:
        print(f'Prediction: {batch_id}/{len(test_dataloader)} completed')

# 인코딩 값으로 나온 타겟 변수를 디코딩
pred = data_test.label_decoder(pred)
print('decode Completed!')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Prediction: 0/97 completed
Prediction: 10/97 completed
Prediction: 20/97 completed
Prediction: 30/97 completed
Prediction: 40/97 completed
Prediction: 50/97 completed
Prediction: 60/97 completed
Prediction: 70/97 completed
Prediction: 80/97 completed
Prediction: 90/97 completed
decode Completed!


In [ ]:
submit = test.copy()
submit

,ID,text
0,0,"""[기자] 지난 24일 국회 국방위원회에서 현역 대령인 기무부대장이 송영무 국방장관..."
1,1,[IMG1] 한예슬-에릭 커플의 드라마 '스파이명월'의 시청률이 3회연속 하락세를 ...
2,2,"""디지털 광장입니다. 운전대를 잡는 시간보다 구르고 넘어지는 시간이 더 많은 극한의..."
3,3,정부가 최저 임금 인상 대책으로 자영업자와 영세사업자에게 지원하는 일자리 안정자금의...
4,4,청와대가 1박2일간 숨가빴던 방북 일정을 마치고 돌아온 대북특별사절단의 후일담을 8...
...,...,...
3098,3098,"중국인 왕즈, 명 밀수 차단에일본 터잡고 ‘왜구 약탈’ 솔선 일 기업인들 성금모아 ..."
3099,3099,"""정상회담을 계기로 남북 학술 교류도 다시 시작될 것으로 보입니다. 멸종된 크낙새를..."
3100,3100,중국 베이징시내 한국 일간신문 정기구독자들에게 18일 배달된 이 날짜 신문에서 자오...
3101,3101,"""인터넷전문은행이 출범한 지 1년이 됐는데요. 창구 갈 필요 없고 복잡한 인증절차도..."


In [ ]:
submit.rename(columns = {'text':'category'}, inplace = True)

In [ ]:
submit

,ID,category
0,0,"""[기자] 지난 24일 국회 국방위원회에서 현역 대령인 기무부대장이 송영무 국방장관..."
1,1,[IMG1] 한예슬-에릭 커플의 드라마 '스파이명월'의 시청률이 3회연속 하락세를 ...
2,2,"""디지털 광장입니다. 운전대를 잡는 시간보다 구르고 넘어지는 시간이 더 많은 극한의..."
3,3,정부가 최저 임금 인상 대책으로 자영업자와 영세사업자에게 지원하는 일자리 안정자금의...
4,4,청와대가 1박2일간 숨가빴던 방북 일정을 마치고 돌아온 대북특별사절단의 후일담을 8...
...,...,...
3098,3098,"중국인 왕즈, 명 밀수 차단에일본 터잡고 ‘왜구 약탈’ 솔선 일 기업인들 성금모아 ..."
3099,3099,"""정상회담을 계기로 남북 학술 교류도 다시 시작될 것으로 보입니다. 멸종된 크낙새를..."
3100,3100,중국 베이징시내 한국 일간신문 정기구독자들에게 18일 배달된 이 날짜 신문에서 자오...
3101,3101,"""인터넷전문은행이 출범한 지 1년이 됐는데요. 창구 갈 필요 없고 복잡한 인증절차도..."


In [ ]:
# 제출 파일 확인
submit['category'] = pred
submit.head(n=10)

,ID,category
0,0,정치
1,1,연예/문화
2,2,생활
3,3,경제
4,4,정치
5,5,경제
6,6,사회
7,7,연예/문화
8,8,연예/문화
9,9,사회


In [ ]:
# 제출 파일 제작
submit.to_csv(os.path.join(ROOT_PATH, 'submit01.csv'), index=False)